In [1]:
from datetime import datetime
import backtrader as bt
%matplotlib notebook

In [ ]:
class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)

cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)

data0 = bt.feeds.YahooFinanceData(dataname='600019.SS', fromdate=datetime(2011, 1, 1),
                                  todate=datetime(2012, 12, 31))
cerebro.adddata(data0)

cerebro.run()
f = cerebro.plot()

In [ ]:
class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        '''Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print '%s, %s' % (dt.isoformat(), txt)

    def __init__(self):
        self.dataclose = self.datas[0].close

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])

        if self.dataclose[-2] > self.dataclose[-1] and self.dataclose[-1] > self.dataclose[0]:
            self.buy()
            self.log('Buy at %.2f' % self.dataclose[0])

In [ ]:
cebebro = bt.Cerebro()

data = bt.feeds.YahooFinanceData(dataname='600019.SS', fromdate=datetime(1900, 1, 1),
                                 todate=date_parser.parse('2000-12-31'), reverse=True)
cebebro.adddata(data)
cebebro.addstrategy(TestStrategy)
cebebro.broker.set_cash(100)



print 'Starting Portfolio value: %.2f' % cebebro.broker.getvalue()

cebebro.run()

print 'Final Portfolio value: %.2f' % cebebro.broker.getvalue()

# Plot

In [16]:
import datetime

import backtrader as bt
from dateutil import parser as date_parser

from matplotlib import pyplot as plt

plt.switch_backend('Qt5Agg')

class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        '''Logging function for this strategy'''
        dt = dt or self.data.datetime.date(0)
        print '%s, %s' % (dt.isoformat(), txt)

    def __init__(self, exitbars=None, maperiod=None):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.params.exitbars = exitbars
        self.params.maperiod = maperiod
        self.sma = bt.indicators.MovingAverageSimple(self.datas[0], period=self.params.maperiod)

        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25, subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=20)
        bt.indicators.ATR(self.datas[0], plot=False)
        bt.indicators.MACDHisto(self.datas[0])

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])

        if self.order:
            return

        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self.order = self.buy()
                self.log('Buy order created at %.2f' % self.dataclose[0])
        else:
            if self.dataclose[0] < self.sma[0]:
                self.log('Sell order created at : %.2f' % self.dataclose[0])
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log('Buy order executed, price: %.2f, cost: %.2f, comm: %.2f' % (order.executed.price,
                                                                                      order.executed.value,
                                                                                      order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            elif order.issell():
                self.log('Sell order executed, price: %.2f, cost: %.2f, comm: %.2f' % (order.executed.price,
                                                                                       order.executed.value,
                                                                                       order.executed.comm))

            self.bar_executed = len(self)

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION profit: Gross: %.2f, Net: %.2f' % (trade.pnl, trade.pnlcomm))

In [17]:
cerebro = bt.Cerebro()

data = bt.feeds.YahooFinanceData(dataname='600019.SS', fromdate=datetime.datetime(2000, 1, 1),
                             todate=date_parser.parse('2001-06-30'), reverse=True)

cerebro.adddata(data)
cerebro.addsizer(bt.sizers.FixedSize, stake=5)
cerebro.addstrategy(TestStrategy, maperiod=15)
cerebro.broker.set_cash(100)
cerebro.broker.setcommission(commission=0.001)
print 'Starting Portfolio value: %.2f' % cerebro.broker.getvalue()

cerebro.run()

print 'Final Portfolio value: %.2f' % cerebro.broker.getvalue()
figure = cerebro.plot()

Starting Portfolio value: 100.00
2001-01-26, Close, 3.77
2001-01-26, Buy order created at 3.77
2001-01-29, Buy order executed, price: 3.78, cost: 18.90, comm: 0.02
2001-01-29, Close, 3.77
2001-01-30, Close, 3.77
2001-01-31, Close, 3.77
2001-02-01, Close, 3.77
2001-02-02, Close, 3.77
2001-02-05, Close, 3.63
2001-02-05, Sell order created at : 3.63
2001-02-06, Sell order executed, price: 3.61, cost: 18.90, comm: 0.02
2001-02-06, OPERATION profit: Gross: -0.85, Net: -0.89
2001-02-06, Close, 3.74
2001-02-07, Close, 3.70
2001-02-08, Close, 3.61
2001-02-09, Close, 3.62
2001-02-12, Close, 3.63
2001-02-13, Close, 3.65
2001-02-14, Close, 3.61
2001-02-15, Close, 3.58
2001-02-16, Close, 3.58
2001-02-19, Close, 3.61
2001-02-20, Close, 3.62
2001-02-21, Close, 3.56
2001-02-22, Close, 3.56
2001-02-23, Close, 3.65
2001-02-23, Buy order created at 3.65
2001-02-26, Buy order executed, price: 3.64, cost: 18.20, comm: 0.02
2001-02-26, Close, 3.60
2001-02-26, Sell order created at : 3.60
2001-02-27, Sell o

In [19]:
fig = plt.figure()
fig.axes.append(figure[0][0].axes[2])
fig.show()

In [23]:
from matplotlib import pyplot as plt
import numpy as np

class GoldenLine():
    
    def __init__(self, cerebo):
        self.golden_pts = np.array([0.236, 0.382, 0.618, 0.809])
        self.figure = cerebo.plot()
        self.axes = self.figure.gca()
        self.figure.canvas.mpl_connect('button_press_event', self.press)
        self.figure.canvas.mpl_connect('button_release_event', self.release)
    
    def press(self, event):
        self.y_start = event.ydata
    
    def release(self, event):
        self.y_end = event.ydata
        self.golden_line()
    
    def golden_line(self):
        high = max(self.y_start, self.y_end)
        low = min(self.y_start, self.y_end)
        middle_pt = low + (high - low) * self.golden_pts
        self.axes.axhline(y=low, linewidth=3, linestyle='-')
        self.axes.axhline(y=high, linewidth=3, linestyle='-')
        for pt in middle_pt:
            self.axes.axhline(y=pt, linewidth=1, linestyle=':')
        self.figure.show()
    
    def start(self):
        self.figure.show()

In [24]:
test = GoldenLine(cerebro)
test.start()

AttributeError: 'list' object has no attribute 'gca'

# Optimization

In [20]:
import datetime

import backtrader as bt
from dateutil import parser as date_parser

from matplotlib import pyplot as plt

plt.switch_backend('Qt5Agg')

class TestStrategy(bt.Strategy):
    params = (('exitbars', None), ('maperiod', None), ('printlog', False))
    
    def log(self, txt, dt=None, doprint=False):
        '''Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print '%s, %s' % (dt.isoformat(), txt)

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.MovingAverageSimple(self.datas[0], period=self.params.maperiod)

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])

        if self.order:
            return

        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self.order = self.buy()
                self.log('Buy order created at %.2f' % self.dataclose[0])
        else:
            if self.dataclose[0] < self.sma[0]:
                self.log('Sell order created at : %.2f' % self.dataclose[0])
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log('Buy order executed, price: %.2f, cost: %.2f, comm: %.2f' % (order.executed.price,
                                                                                      order.executed.value,
                                                                                      order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            elif order.issell():
                self.log('Sell order executed, price: %.2f, cost: %.2f, comm: %.2f' % (order.executed.price,
                                                                                       order.executed.value,
                                                                                       order.executed.comm))

            self.bar_executed = len(self)

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION profit: Gross: %.2f, Net: %.2f' % (trade.pnl, trade.pnlcomm))
        
    def stop(self):
        self.log('MA Period: %2d Ending Value: %.2f' % (self.params.maperiod, self.broker.getvalue()),
                 doprint=True)

In [25]:
cerebro = bt.Cerebro()

data = bt.feeds.YahooFinanceData(dataname='600019.SS', fromdate=datetime.datetime(2000, 1, 1),
                             todate=date_parser.parse('2001-06-30'), reverse=True)

cerebro.adddata(data)
cerebro.addsizer(bt.sizers.FixedSize, stake=5)
strats = cerebro.optstrategy(TestStrategy, maperiod=range(10,31))
cerebro.broker.set_cash(100)
cerebro.broker.setcommission(commission=0.001)
result = cerebro.run(optreturn=False)

2001-06-29, MA Period: 10 Ending Value: 101.24
2001-06-29, MA Period: 12 Ending Value: 100.42
2001-06-29, MA Period: 13 Ending Value: 100.34
2001-06-29, MA Period: 11 Ending Value: 100.30
2001-06-29, MA Period: 16 Ending Value: 99.85
2001-06-29, MA Period: 15 Ending Value: 99.85
2001-06-29, MA Period: 14 Ending Value: 100.19
2001-06-29, MA Period: 17 Ending Value: 99.16
2001-06-29, MA Period: 18 Ending Value: 99.01
2001-06-29, MA Period: 19 Ending Value: 99.42
2001-06-29, MA Period: 20 Ending Value: 98.45
2001-06-29, MA Period: 22 Ending Value: 97.97
2001-06-29, MA Period: 21 Ending Value: 98.40
2001-06-29, MA Period: 23 Ending Value: 98.50
2001-06-29, MA Period: 24 Ending Value: 97.62
2001-06-29, MA Period: 25 Ending Value: 98.80
2001-06-29, MA Period: 26 Ending Value: 97.50
2001-06-29, MA Period: 27 Ending Value: 97.60
2001-06-29, MA Period: 28 Ending Value: 97.29
2001-06-29, MA Period: 30 Ending Value: 97.84
2001-06-29, MA Period: 29 Ending Value: 97.54


In [29]:
(result[0][0]).bar_executed

140